<a href="https://colab.research.google.com/github/AmitKPandey11/100-plus-python-coding-problems-with-solutions/blob/master/Bunkalab_Setfit_text_classification_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U transformers==4.39.0
!pip install -q -U datasets==2.18.0
!pip install -q -U setfit==1.0.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 808.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 48.8 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer

In [3]:
from datasets import load_dataset

model_id = "sentence-transformers/paraphrase-mpnet-base-v2"
dataset = load_dataset("bunkalab/medium-sample-technology-tags")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1394 [00:00<?, ? examples/s]

In [4]:
dataset["train"].column_names

['title', 'tags', 'doc_id']

In [5]:
dataset["train"][0]

{'title': 'Efficient energy storage and future PVS by Solar DAO',
 'tags': 'Blockchain',
 'doc_id': 3}

In [6]:
import pandas as pd

df = pd.DataFrame(dataset["train"])

In [7]:
# Find top classes with minimum 30 rows
label_counts = df["tags"].value_counts()
    #label_counts = label_counts[label_counts > 30]

In [8]:
label_counts

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tags
Blockchain                 248
Programming                218
Artificial Intelligence    170
Startup                    166
Tech                       154
Software Development       146
Blockchain Technology      117
Data Science                97
Cryptocurrency              78
Name: count, dtype: int64

In [9]:

import pandas as pd
from datasets import Dataset, load_dataset

from setfit import SetFitModel, Trainer, TrainingArguments
from sklearn.metrics import classification_report
from tqdm import tqdm
from transformers import AutoTokenizer, pipeline

In [30]:
# Helper function to select random rows
def select_random_rows(group):
    return group.sample(n=30, random_state=42)

In [31]:
# Find top classes with minimum 30 rows
label_counts = df["tags"].value_counts()
label_counts = label_counts[label_counts > 50]

    # Restrict df to top classes only
df = df[df["tags"].isin(label_counts.index)].reset_index(drop=True)
assert set(df["tags"].value_counts().index.to_list()) == set(
    label_counts.index.to_list()
), "Some labels were lost"

# Select random rows per unique value in label_text column
train_df = df.groupby("tags", group_keys=False).apply(select_random_rows)

# Create eval dataframe by dropping the train data and selecting random rows
eval_df = (
    df.drop(train_df.index)
    .groupby("tags", group_keys=False)
    .apply(select_random_rows)
)

# Create test dataframe by dropping both train and eval data
test_df = df.drop(train_df.index.to_list() + eval_df.index.to_list())

# Reset the index
cols_to_keep = ["title", "tags"]
train_df = train_df[cols_to_keep].reset_index(drop=True)
eval_df = eval_df[cols_to_keep].reset_index(drop=True)
test_df = test_df[cols_to_keep].reset_index(drop=True)

# Save the file
test_df.to_pickle("test_df.pkl")
train_df.to_pickle("train_df.pkl")
eval_df.to_pickle("eval_df.pkl")

# Convert the pandas DataFrame to Hugging Face's Dataset
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)
test_dataset = Dataset.from_pandas(test_df)

print(train_df.shape, eval_df.shape, test_df.shape)
train_df.head()

(270, 2) (270, 2) (854, 2)


,title,tags
0,Raspberry Pi : Agent of Change. The Raspberry ...,Artificial Intelligence
1,Can Artificial Intelligence Make Us Safer?,Artificial Intelligence
2,Amazon’s First Non-Employee Customer and What ...,Artificial Intelligence
3,How 5G will change the world,Artificial Intelligence
4,The Top Online Data Science Courses for 2019,Artificial Intelligence


In [32]:
# Step 1: Load the base ST model
model = SetFitModel.from_pretrained(
    "sentence-transformers/paraphrase-mpnet-base-v2",
    labels=train_df["tags"].unique().tolist(),
)

# Step 2: Setup the training
args = TrainingArguments(
    batch_size=16,
    num_epochs=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

args.evaluation_strategy==args.evaluation_strategy

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    metric="accuracy",
    # column_mapping is {"custom dataset column name": "setfit expected column name (text or label)"}
    column_mapping={
        "title": "text",
        "tags": "label",
    },
)

# Step 3: Train the model!
trainer.train()

# Step 4: Save the trained model to disk
model.save_pretrained(
    "bunkalab/medium-sample-technology-tags"
)

# Step 5: Calculate metrics on test dataset
metrics = trainer.evaluate(test_dataset)
print("Accuracy: {:.2f}%".format(metrics["accuracy"] * 100))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


Map:   0%|          | 0/270 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 64800
  Batch size = 16
  Num epochs = 4
  Total optimization steps = 16200


Epoch,Training Loss,Validation Loss,Embedding Loss,Rate
1,No log,No log,0.347800,0.000017
2,No log,No log,0.359700,0.000011
3,No log,No log,0.372700,0.000006
4,No log,No log,0.378900,0.000000


  0%|          | 0/4050 [00:00<?, ?it/s]

  0%|          | 0/4050 [00:00<?, ?it/s]

  0%|          | 0/4050 [00:00<?, ?it/s]

  0%|          | 0/4050 [00:00<?, ?it/s]

Loading best SentenceTransformer model from step 4050.
Applying column mapping to the evaluation dataset
***** Running evaluation *****


Accuracy: 41.22%


In [33]:
# Step 1: Load a pre-trained model from disk
model = SetFitModel.from_pretrained(
    "bunkalab/medium-sample-technology-tags"
)

# Step 2: Run predictions to calculate class level metrics
test_df["predictions"] = model.predict(test_df["title"].values)

print("Class Level Metrics:")
print(classification_report(test_df["tags"], test_df["predictions"]))

Class Level Metrics:
                         precision    recall  f1-score   support

Artificial Intelligence       0.65      0.48      0.55       110
             Blockchain       0.62      0.43      0.50       188
  Blockchain Technology       0.19      0.30      0.23        57
         Cryptocurrency       0.11      0.22      0.15        18
           Data Science       0.23      0.49      0.31        37
            Programming       0.47      0.39      0.43       158
   Software Development       0.30      0.38      0.34        86
                Startup       0.45      0.46      0.45       106
                   Tech       0.42      0.38      0.40        94

               accuracy                           0.41       854
              macro avg       0.38      0.39      0.37       854
           weighted avg       0.46      0.41      0.43       854



In [16]:
# Step 1: Load a pre-trained model from disk
model = SetFitModel.from_pretrained(
    "bunkalab/medium-sample-technology-tags"
)

# Step 2: Run predictions to calculate class level metrics
test_df["predictions"] = model.predict(test_df["title"].values)

print("Class Level Metrics:")
print(classification_report(test_df["tags"], test_df["predictions"]))

Class Level Metrics:
                         precision    recall  f1-score   support

Artificial Intelligence       0.48      0.41      0.44       150
             Blockchain       0.64      0.45      0.53       228
  Blockchain Technology       0.22      0.16      0.19        97
         Cryptocurrency       0.16      0.60      0.26        58
           Data Science       0.41      0.53      0.47        77
            Programming       0.41      0.39      0.40       198
   Software Development       0.28      0.24      0.26       126
                Startup       0.35      0.38      0.36       146
                   Tech       0.47      0.31      0.37       134

               accuracy                           0.38      1214
              macro avg       0.38      0.39      0.36      1214
           weighted avg       0.42      0.38      0.39      1214

